In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


# Configuration

In [3]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv
import random
import copy

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
config = {
    'realworld': {
        'dataset_name': 'realworld',
        'num_df_domains': 10,
        'num_dp_domains': 5,
        'num_classes': 4,
        'class_names': ['WAL', 'RUN', 'CLD', 'CLU'],
        'num_timesteps': 128,
        'num_channels': 3,
        'num_classes': 4,
    },
    'cwru': {
        'dataset_name': 'cwru_256_3ch_5cl',
        'num_df_domains': 4,
        'num_dp_domains': 4,
        'num_classes': 5,
        'class_names': ['IR', 'Ball', 'OR_centred', 'OR_orthogonal', 'OR_opposite'],
        'num_timesteps': 256,
        'num_channels': 3,
        'num_classes': 5,
    },
    'realworld_mobiact': {
        'dataset_name': 'realworld_mobiact',
        'num_df_domains': 15,
        'num_dp_domains': 61,
        'num_classes': 4,
        'class_names': ['WAL', 'RUN', 'CLD', 'CLU'],
        'num_timesteps': 128,
        'num_channels': 3,
        'num_classes': 4,
    }
}

syn_name = 'rwma11'
num_epochs = 100

In [ ]:
def get_data(dataset, domains_set, src_class, domain=None, rot=False):
    # Load configurations
    dataset_name = config[dataset]['dataset_name']
    class_idx = config[dataset]['class_names'].index(src_class)
    num_df_domains = config[dataset]['num_df_domains']

    if rot:
        dataset_name += '_rot'

    try:
        with open(f'data/{dataset_name}.pkl', 'rb') as f:
            x, y, k = pickle.load(f)
        with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
            fs = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")


    if domains_set == 'df':
        mask = (y != class_idx) & (k < num_df_domains)
    elif domains_set == 'dp':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 0)
    elif domains_set == 'dpfs':
        mask = (y != class_idx) & (k >= num_df_domains) & (fs == 1)
    else:
        raise ValueError(f"Invalid domains set: {domains_set}")

    # Apply initial mask
    x, y, k = x[mask], y[mask], k[mask]

    # Additional domain filtering if specified
    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



def get_syn_data(dataset, syn_name, src_class, domain=None, fs=False):
    # Load configurations
    class_names = config[dataset]['class_names']

    try:
        with open(f'data/{dataset}_{syn_name}.pkl', 'rb') as f:
            x, y, k = pickle.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Dataset files for {dataset} not found.")

    with open(f'data/{dataset}_{syn_name}_fs.pkl', 'rb') as f:
        fs = pickle.load(f)

    x, y, k = x[fs == 0], y[fs == 0], k[fs == 0]

    if domain is not None:
        domain_mask = (k == domain)
        x, y, k = x[domain_mask], y[domain_mask], k[domain_mask]

    assert len(x) > 0, f"No data found"
    assert len(x) == len(y) == len(k), f"Data length mismatch"

    return x, y, k



class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=5):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs



def remap_labels(y):
    label_map = {clss: i for i, clss in enumerate(np.unique(y))}
    return np.array([label_map[clss] for clss in y])



def get_dataloader(x, y, shuffle=False):
    x = torch.tensor(x, dtype=torch.float32)
    y = remap_labels(y)
    y = torch.tensor(y, dtype=torch.long)

    # Determine dataset size and batch size
    dataset_size = len(x)
    if dataset_size <= 100:
        batch_size = dataset_size
    elif dataset_size <= 1000:
        batch_size = 16
    elif dataset_size <= 5000:
        batch_size = 32
    else:
        batch_size = 64

    dataset = TensorDataset(x, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return loader


def random_rotation_matrix():
    """Generate a random rotation matrix from a predefined set of quaternions."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Randomly generate a quaternion
    q = np.random.rand(4)

    # Convert quaternion to rotation matrix
    q = torch.tensor(q, device=device, dtype=torch.float32)
    q = q / torch.norm(q)  # Normalize quaternion
    q0, q1, q2, q3 = q

    R = torch.tensor([
        [1 - 2*q2**2 - 2*q3**2, 2*q1*q2 - 2*q3*q0, 2*q1*q3 + 2*q2*q0],
        [2*q1*q2 + 2*q3*q0, 1 - 2*q1**2 - 2*q3**2, 2*q2*q3 - 2*q1*q0],
        [2*q1*q3 - 2*q2*q0, 2*q2*q3 + 2*q1*q0, 1 - 2*q1**2 - 2*q2**2]
    ], device=device, dtype=torch.float32)

    return R, q


def augment_batch(x_real):
    """Apply random rotation to the batch of real time series."""
    min_val, max_val = -19.61, 19.61
    x_real = x_real * (max_val - min_val) + min_val  # De-normalize
    R, q = random_rotation_matrix()
    x_real = torch.matmul(R, x_real)  # Apply rotation
    x_real = (x_real - min_val) / (max_val - min_val)  # Re-normalize
    return x_real, q



def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    loss_fn = nn.CrossEntropyLoss()

    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs, 1)
            all_preds.extend(predicted_labels.detach().cpu().numpy())
            all_labels.extend(y_batch.detach().cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    total_loss /= len(test_loader)

    return accuracy, total_loss, f1



def train_model(model, train_loader, val_loader, optimizer, num_epochs=100, augment=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_fn = nn.CrossEntropyLoss()

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_model_state = None
    best_loss = np.inf
    best_accuracy = 0
    best_f1 = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / num_epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            if augment:
                x_batch, _ = augment_batch(x_batch)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss, val_f1 = evaluate_model(model, val_loader)
        if val_loss < best_loss:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_f1 = val_f1
            best_loss = val_loss
            best_model_state = model.state_dict().copy()

        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        if (epoch+1) % 10 == 0:
            print(f"\tEpoch {epoch + 1}/{num_epochs} - Train loss: {total_loss:.4f} - Val accuracy: {val_accuracy:.4f} - Val loss: {val_loss:.4f} - Val F1: {val_f1:.4f} - LR: {current_lr:.2e}")

    print(f"\tBest epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f} - Best val F1: {best_f1:.4f}\n")

    # Load best model state
    model.load_state_dict(best_model_state)

    return model



def train_and_test(x_train, y_train, x_test, y_test, dataset, num_epochs=100, augment=False):
    assert np.array_equal(np.unique(y_train), np.unique(y_test)), "Training and test labels do not match"

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)
    test_loader = get_dataloader(x_test, y_test)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs, augment)

    test_accuracy, test_loss, test_f1 = evaluate_model(trained_model, test_loader)

    return test_accuracy, test_loss, test_f1



def train_only(x_train, y_train, dataset, num_epochs=100, augment=False):

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    model = TSTRClassifier(num_timesteps=config[dataset]['num_timesteps'],
                           num_channels=config[dataset]['num_channels'],
                           num_classes=config[dataset]['num_classes']-1)
    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs, augment=augment)

    return trained_model



def train_classifier_cv(x_train, y_train, dataset, num_epochs=100):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    accs = []
    losses = []
    f1s = []

    for train_index, test_index in skf.split(x_train, y_train):
        x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        acc, loss, f1 = train_and_test(x_train_fold, y_train_fold, x_test_fold, y_test_fold, dataset, num_epochs)
        accs.append(acc)
        losses.append(loss)
        f1s.append(f1)
    return np.mean(accs), np.mean(losses), np.mean(f1s)



def fine_tune(model, x_train, y_train, num_epochs=100):
    # Freeze feature extraction layers
    for name, param in model.named_parameters():
        if 'conv' in name or 'bn' in name:
            param.requires_grad = False

    x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=seed)

    train_loader = get_dataloader(x_tr, y_tr, shuffle=True)
    val_loader = get_dataloader(x_val, y_val)

    initial_lr = 0.00001
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=initial_lr)

    trained_model = train_model(model, train_loader, val_loader, optimizer, num_epochs)

    return trained_model



def save_scores(source, domain, accuracy, loss, f1, name, dataset):
    results_dir = 'results'
    # Ensure the directory exists
    os.makedirs(results_dir, exist_ok=True)
    # Path to the CSV file
    file_path = os.path.join(results_dir, f'{dataset}_{name}.csv')
    # Check if the file exists
    file_exists = os.path.exists(file_path)

    # Open the file in append mode if it exists, or write mode if it doesn't
    with open(file_path, mode='a' if file_exists else 'w', newline='') as file:
        writer = csv.writer(file)
        # If the file does not exist, write the header
        if not file_exists:
            writer.writerow(['source', 'domain', 'accuracy', 'loss', 'f1'])
        # Write the data rows
        writer.writerow([source, domain, accuracy, loss, f1])

# TSTR

In [6]:
def compute_TSTR_Dp(dataset):
    accs = []
    f1s = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            if domain == 74:
              continue

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train and evaluate via cross-validation on Dp data
            print('Training and evaluating on Dp data via cross-validation...')
            acc, loss, f1 = train_classifier_cv(x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, f1, 'Dp', dataset)
            accs.append(acc)
            f1s.append(f1)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}")
    print(f"Mean F1: {np.mean(f1s):.4f}\n")



compute_TSTR_Dp('realworld_mobiact')

Source class: WAL

Domain: 15
x_dp_dom.shape: (50, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training and evaluating on Dp data via cross-validation...
	Epoch 10/100 - Train loss: 0.6204 - Val accuracy: 0.2500 - Val loss: 1.1244 - Val F1: 0.1000 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.3222 - Val accuracy: 0.2500 - Val loss: 1.1118 - Val F1: 0.1000 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.1628 - Val accuracy: 0.3750 - Val loss: 1.1155 - Val F1: 0.2778 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.1092 - Val accuracy: 0.5000 - Val loss: 0.9516 - Val F1: 0.3750 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0749 - Val accuracy: 1.0000 - Val loss: 0.4202 - Val F1: 1.0000 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0541 - Val accuracy: 1.0000 - Val loss: 0.1393 - Val F1: 1.0000 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0436 - Val accuracy: 1.0000 - Val loss: 0.0767 - Val F1: 1.0000 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0414 - Val accuracy: 1.0000 - Val loss: 0.0599 - Val F1: 

In [7]:
def compute_TSTR_Dpc(dataset):

    raise NotImplementedError



# compute_TSTR_Dpc('realworld_mobiact')

In [8]:
n_runs = 10

def compute_TSTR_Df(dataset):
    accs = []
    f1s = []

    for i in range(n_runs):

        accs_run = []
        f1s_run = []

        for src_class in config[dataset]['class_names']:
            if src_class != 'WAL':
                continue

            print(f"Source class: {src_class}\n")

            # Load Df data
            x_df, y_df, k_df = get_data(dataset, 'df', src_class)
            print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

            # Train on Df data
            print('Training on Df data...')
            df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

            for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
                print(f"Domain: {domain}")

                # Load Dp data
                x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
                print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

                # Evaluate on Dp data
                acc, loss, f1 = evaluate_model(df_model, get_dataloader(x_dp_dom, y_dp_dom))
                save_scores(src_class, domain, acc, loss, f1, 'Df', dataset)
                accs_run.append(acc)
                f1s_run.append(f1)
                print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

        print(f"Mean accuracy for run {i}: {np.mean(accs_run):.4f}")
        print(f"Mean F1 for run {i}: {np.mean(f1s_run):.4f}\n")
        accs.append(np.mean(accs_run))
        f1s.append(np.mean(f1s_run))

    print(f"Mean accuracy over {n_runs} runs: {np.mean(accs):.4f} +- {np.std(accs):.4f}")
    print(f"Mean F1 over {n_runs} runs: {np.mean(f1s):.4f} +- {np.std(f1s):.4f}\n")




compute_TSTR_Df('realworld_mobiact')

Source class: WAL

x_df.shape: (7958, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0314 - Val accuracy: 0.9812 - Val loss: 0.0523 - Val F1: 0.9811 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0095 - Val accuracy: 0.9887 - Val loss: 0.0342 - Val F1: 0.9887 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0040 - Val accuracy: 0.9906 - Val loss: 0.0311 - Val F1: 0.9906 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0037 - Val accuracy: 0.9899 - Val loss: 0.0344 - Val F1: 0.9900 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0012 - Val accuracy: 0.9899 - Val loss: 0.0359 - Val F1: 0.9899 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0006 - Val accuracy: 0.9912 - Val loss: 0.0384 - Val F1: 0.9912 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0005 - Val accuracy: 0.9899 - Val loss: 0.0397 - Val F1: 0.9899 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0004 - Val accuracy: 0.9925 - Val loss: 0.0378 - Val F1: 0.9925 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0

In [9]:
n_runs = 10

def compute_TSTR_Df_aug(dataset):
    accs = []
    f1s = []

    for i in range(n_runs):

        accs_run = []
        f1s_run = []

        for src_class in config[dataset]['class_names']:
            if src_class != 'WAL':
                continue

            print(f"Source class: {src_class}\n")

            # Load Df data
            x_df, y_df, k_df = get_data(dataset, 'df', src_class)
            print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

            # Train on Df data
            print('Training on Df data...')
            df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs, augment=True)

            for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
                print(f"Domain: {domain}")

                # Load Dp data
                x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
                print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

                # Evaluate on Dp data
                acc, loss, f1 = evaluate_model(df_model, get_dataloader(x_dp_dom, y_dp_dom))
                save_scores(src_class, domain, acc, loss, f1, 'Df_aug', dataset)
                accs_run.append(acc)
                f1s_run.append(f1)
                print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

        print(f"Mean accuracy for run {i}: {np.mean(accs_run):.4f}")
        print(f"Mean F1 for run {i}: {np.mean(f1s_run):.4f}\n")
        accs.append(np.mean(accs_run))
        f1s.append(np.mean(f1s_run))

    print(f"Mean accuracy over {n_runs} runs: {np.mean(accs):.4f} +- {np.std(accs):.4f}")
    print(f"Mean F1 over {n_runs} runs: {np.mean(f1s):.4f} +- {np.std(f1s):.4f}\n")




compute_TSTR_Df_aug('realworld_mobiact')

Source class: WAL

x_df.shape: (7958, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.1089 - Val accuracy: 0.7682 - Val loss: 0.5951 - Val F1: 0.7782 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0741 - Val accuracy: 0.8247 - Val loss: 0.4775 - Val F1: 0.8252 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0695 - Val accuracy: 0.7701 - Val loss: 0.6668 - Val F1: 0.7509 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0571 - Val accuracy: 0.7280 - Val loss: 1.0030 - Val F1: 0.6242 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0530 - Val accuracy: 0.7305 - Val loss: 1.1198 - Val F1: 0.6612 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0401 - Val accuracy: 0.7977 - Val loss: 0.6788 - Val F1: 0.7822 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0424 - Val accuracy: 0.6263 - Val loss: 1.7234 - Val F1: 0.5996 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0366 - Val accuracy: 0.7538 - Val loss: 1.0877 - Val F1: 0.7065 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0

In [10]:
n_runs = 10

def compute_TSTR_Df_rot(dataset):
    accs = []
    f1s = []

    for i in range(n_runs):

        accs_run = []
        f1s_run = []

        for src_class in config[dataset]['class_names']:
            if src_class != 'WAL':
                continue

            print(f"Source class: {src_class}\n")

            # Load Df data
            x_df, y_df, k_df = get_data(dataset, 'df', src_class, rot=True)
            print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

            # Train on Df data
            print('Training on Df data...')
            df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

            for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
                print(f"Domain: {domain}")

                # Load Dp data
                x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
                print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

                # Evaluate on Dp data
                acc, loss, f1 = evaluate_model(df_model, get_dataloader(x_dp_dom, y_dp_dom))
                save_scores(src_class, domain, acc, loss, f1, 'Df_rot', dataset)
                accs_run.append(acc)
                f1s_run.append(f1)
                print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

        print(f"Mean accuracy for run {i}: {np.mean(accs_run):.4f}")
        print(f"Mean F1 for run {i}: {np.mean(f1s_run):.4f}\n")
        accs.append(np.mean(accs_run))
        f1s.append(np.mean(f1s_run))

    print(f"Mean accuracy over {n_runs} runs: {np.mean(accs):.4f} +- {np.std(accs):.4f}")
    print(f"Mean F1 over {n_runs} runs: {np.mean(f1s):.4f} +- {np.std(f1s):.4f}\n")




compute_TSTR_Df_rot('realworld_mobiact')

Source class: WAL

x_df.shape: (7958, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0310 - Val accuracy: 0.9868 - Val loss: 0.0454 - Val F1: 0.9868 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0091 - Val accuracy: 0.9856 - Val loss: 0.0438 - Val F1: 0.9856 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0033 - Val accuracy: 0.9868 - Val loss: 0.0517 - Val F1: 0.9868 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0013 - Val accuracy: 0.9868 - Val loss: 0.0585 - Val F1: 0.9868 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0009 - Val accuracy: 0.9874 - Val loss: 0.0540 - Val F1: 0.9874 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0029 - Val accuracy: 0.9862 - Val loss: 0.0566 - Val F1: 0.9862 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0003 - Val accuracy: 0.9874 - Val loss: 0.0522 - Val F1: 0.9874 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0002 - Val accuracy: 0.9874 - Val loss: 0.0558 - Val F1: 0.9874 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0

In [11]:
def compute_TSTR_Syn(dataset):
    accs = []
    f1s = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data and evaluate on Dp data
            print('Training on synthetic data...')
            acc, loss, f1 = train_and_test(x_syn_dom, y_syn_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, f1, 'Syn', dataset)
            accs.append(acc)
            f1s.append(f1)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}")
    print(f"Mean F1: {np.mean(f1s):.4f}\n")




compute_TSTR_Syn('realworld_mobiact')

Source class: WAL

Domain: 15
x_syn_dom.shape: (333, 3, 128) | np.unique(y_syn_dom): [1 2 3]
x_dp_dom.shape: (50, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on synthetic data...
	Epoch 10/100 - Train loss: 0.1386 - Val accuracy: 0.9851 - Val loss: 0.1310 - Val F1: 0.9851 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0182 - Val accuracy: 0.9851 - Val loss: 0.0201 - Val F1: 0.9851 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0050 - Val accuracy: 1.0000 - Val loss: 0.0143 - Val F1: 1.0000 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0027 - Val accuracy: 0.9851 - Val loss: 0.0174 - Val F1: 0.9851 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0014 - Val accuracy: 1.0000 - Val loss: 0.0090 - Val F1: 1.0000 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0022 - Val accuracy: 1.0000 - Val loss: 0.0099 - Val F1: 1.0000 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0022 - Val accuracy: 0.9851 - Val loss: 0.0121 - Val F1: 0.9851 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0010 - Val accuracy: 1.

In [ ]:
def compute_TSTR_Syn_aug(dataset):
    accs = []
    f1s = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data and evaluate on Dp data
            print('Training on synthetic data...')
            acc, loss, f1 = train_and_test(x_syn_dom, y_syn_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs, augment=True)
            save_scores(src_class, domain, acc, loss, f1, 'Syn_aug', dataset)
            accs.append(acc)
            f1s.append(f1)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}")
    print(f"Mean F1: {np.mean(f1s):.4f}\n")




compute_TSTR_Syn_aug('realworld_mobiact')

Source class: WAL

Domain: 15
x_syn_dom.shape: (333, 3, 128) | np.unique(y_syn_dom): [1 2 3]
x_dp_dom.shape: (50, 3, 128) | np.unique(y_dp_dom): [1 2 3]

Training on synthetic data...
	Epoch 10/100 - Train loss: 0.3227 - Val accuracy: 0.3433 - Val loss: 3.9059 - Val F1: 0.1755 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0871 - Val accuracy: 0.3433 - Val loss: 4.1578 - Val F1: 0.1755 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0436 - Val accuracy: 0.3433 - Val loss: 5.2888 - Val F1: 0.1755 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0457 - Val accuracy: 0.3433 - Val loss: 3.5049 - Val F1: 0.1755 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0337 - Val accuracy: 0.3433 - Val loss: 5.6195 - Val F1: 0.1755 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0213 - Val accuracy: 0.3433 - Val loss: 6.1638 - Val F1: 0.1755 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0337 - Val accuracy: 0.3433 - Val loss: 4.6334 - Val F1: 0.1755 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0129 - Val accuracy: 0.

In [13]:
# def compute_TSTR_Syn_all(dataset):
#     accs = []

#     for src_class in config[dataset]['class_names']:
#         if src_class != 'WAL':
#             continue

#         print(f"Source class: {src_class}\n")

#         # Load synthetic data
#         x_syn, y_syn, k_syn = get_syn_data(dataset, syn_name, src_class)
#         print(f'x_syn.shape: {x_syn.shape} | np.unique(y_syn): {np.unique(y_syn)}')

#         # Load Dp data
#         x_dp, y_dp, k_dp = get_data(dataset, 'dp', src_class)
#         print(f'x_dp.shape: {x_dp.shape} | np.unique(y_dp): {np.unique(y_dp)}\n')

#         # Train on synthetic data and evaluate on Dp data
#         print('Training on synthetic data...')
#         acc, loss = train_and_test(x_syn, y_syn, x_dp, y_dp, dataset, num_epochs=num_epochs)
#         save_scores(src_class, 100, acc, loss, 'Syn_all', dataset)
#         accs.append(acc)
#         print(f'Source class: {src_class} | Domain: All | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

#     print(f"Mean accuracy: {np.mean(accs):.4f}\n")




# compute_TSTR_Syn_all('realworld_mobiact')

In [14]:
def compute_TSTR_Df_Syn(dataset):
    accs = []
    f1s = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data and evaluate on Dp data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=num_epochs)
            acc, loss, f1 = evaluate_model(df_syn_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, f1, 'Df_Syn', dataset)
            accs.append(acc)
            f1s.append(f1)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f} | F1: {f1:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}")
    print(f"Mean F1: {np.mean(f1s):.4f}\n")




compute_TSTR_Df_Syn('realworld_mobiact')

Source class: WAL

x_df.shape: (7958, 3, 128) | np.unique(y_df): [1 2 3]

Training on Df data...
	Epoch 10/100 - Train loss: 0.0261 - Val accuracy: 0.9849 - Val loss: 0.0419 - Val F1: 0.9849 - LR: 9.00e-05
	Epoch 20/100 - Train loss: 0.0108 - Val accuracy: 0.9868 - Val loss: 0.0473 - Val F1: 0.9868 - LR: 8.00e-05
	Epoch 30/100 - Train loss: 0.0032 - Val accuracy: 0.9874 - Val loss: 0.0387 - Val F1: 0.9874 - LR: 7.00e-05
	Epoch 40/100 - Train loss: 0.0022 - Val accuracy: 0.9856 - Val loss: 0.0448 - Val F1: 0.9855 - LR: 6.00e-05
	Epoch 50/100 - Train loss: 0.0012 - Val accuracy: 0.9874 - Val loss: 0.0447 - Val F1: 0.9874 - LR: 5.00e-05
	Epoch 60/100 - Train loss: 0.0005 - Val accuracy: 0.9874 - Val loss: 0.0471 - Val F1: 0.9874 - LR: 4.00e-05
	Epoch 70/100 - Train loss: 0.0006 - Val accuracy: 0.9874 - Val loss: 0.0486 - Val F1: 0.9874 - LR: 3.00e-05
	Epoch 80/100 - Train loss: 0.0003 - Val accuracy: 0.9874 - Val loss: 0.0463 - Val F1: 0.9874 - LR: 2.00e-05
	Epoch 90/100 - Train loss: 0.0

In [15]:
# def compute_TSTR_Df_Syn_all(dataset):
#     accs = []

#     for src_class in config[dataset]['class_names']:
#         if src_class != 'WAL':
#             continue

#         print(f"Source class: {src_class}\n")

#         # Load Df data
#         x_df, y_df, k_df = get_data(dataset, 'df', src_class)
#         print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

#         # Train on Df data
#         print('Training on Df data...')
#         df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

#         # Load synthetic data
#         x_syn, y_syn, k_syn = get_syn_data(dataset, syn_name, src_class)
#         print(f'x_syn.shape: {x_syn.shape} | np.unique(y_syn): {np.unique(y_syn)}')

#         # Load Dp data
#         x_dp, y_dp, k_dp = get_data(dataset, 'dp', src_class)
#         print(f'x_dp.shape: {x_dp.shape} | np.unique(y_dp): {np.unique(y_dp)}\n')

#         # Fine-tune Df model on synthetic data and evaluate on Dp data
#         print('Fine-tuning on synthetic data...')
#         df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn, y_syn, num_epochs=num_epochs)
#         acc, loss = evaluate_model(df_syn_model, get_dataloader(x_dp, y_dp))
#         save_scores(src_class, 100, acc, loss, 'Df_Syn_all', dataset)
#         accs.append(acc)
#         print(f'Source class: {src_class} | Domain: All | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

#     print(f"Mean accuracy: {np.mean(accs):.4f}\n")




# compute_TSTR_Df_Syn_all('realworld_mobiact')

# TSTRFS

In [ ]:
def compute_TSTRFS_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Dpfs data and test on Dp data
            print('Training on Dpfs data...')
            acc, loss = train_and_test(x_dpfs_dom, y_dpfs_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, 'FS_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Dpfs('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Df_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on Dpfs data and test on Dp data
            print('Fine-tuning on Dpfs data...')
            df_dpfs_model = fine_tune(copy.deepcopy(df_model), x_dpfs_dom, y_dpfs_dom, num_epochs=num_epochs)
            acc, loss = evaluate_model(df_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Df_Dpfs('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data and evaluate on Dp data
            print('Training on synthetic data...')
            acc, loss = train_and_test(x_syn_dom, y_syn_dom, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, 'FS_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Df_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data and evaluate on Dp data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=num_epochs)
            acc, loss = evaluate_model(df_syn_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on synthetic data
            print('Training on synthetic data...')
            syn_model = train_only(x_syn_dom, y_syn_dom, dataset, num_epochs=num_epochs)

            # Fine-tune synthetic model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            syn_dpfs_model = fine_tune(copy.deepcopy(syn_model), x_dpfs_dom, y_dpfs_dom, num_epochs=num_epochs)
            acc, loss = evaluate_model(syn_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Syn_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Syn_Dpfs('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Df_Syn_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        # Train on Df data
        print('Training on Df data...')
        df_model = train_only(x_df, y_df, dataset, num_epochs=num_epochs)

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Fine-tune Df model on synthetic data
            print('Fine-tuning on synthetic data...')
            df_syn_model = fine_tune(copy.deepcopy(df_model), x_syn_dom, y_syn_dom, num_epochs=num_epochs)

            # Fine-tune Df-syn model on Dpfs data and evaluate on Dp data
            print('Fine-tuning on Dpfs data...')
            df_syn_dpfs_model = fine_tune(copy.deepcopy(df_syn_model), x_dpfs_dom, y_dpfs_dom, num_epochs=num_epochs)
            acc, loss = evaluate_model(df_syn_dpfs_model, get_dataloader(x_dp_dom, y_dp_dom))
            save_scores(src_class, domain, acc, loss, 'FS_Df_Syn_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_Syn_Dpfs('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Df_plus_Dpfs(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load Dpfs data
            x_dpfs_dom, y_dpfs_dom, k_dpfs_dom = get_data(dataset, 'dpfs', src_class, domain)
            print(f'x_dpfs_dom.shape: {x_dpfs_dom.shape} | np.unique(y_dpfs_dom): {np.unique(y_dpfs_dom)}')

            # Join Df and Dpfs data
            x_df_dpfs = np.concatenate([x_df, x_dpfs_dom], axis=0)
            y_df_dpfs = np.concatenate([y_df, y_dpfs_dom], axis=0)
            k_df_dpfs = np.concatenate([k_df, k_dpfs_dom], axis=0)
            print(f'x_df_dpfs.shape: {x_df_dpfs.shape} | np.unique(y_df_dpfs): {np.unique(y_df_dpfs)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Df plus Dpfs data and test on Dp data
            print('Training on Df plus Dpfs data...')
            acc, loss = train_and_test(x_df_dpfs, y_df_dpfs, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, 'FS_Df_plus_Dpfs', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")



compute_TSTRFS_Df_plus_Dpfs('realworld_mobiact')

In [ ]:
def compute_TSTRFS_Df_plus_Syn(dataset):
    accs = []

    for src_class in config[dataset]['class_names']:
        if src_class != 'WAL':
            continue

        print(f"Source class: {src_class}\n")

        # Load Df data
        x_df, y_df, k_df = get_data(dataset, 'df', src_class)
        print(f'x_df.shape: {x_df.shape} | np.unique(y_df): {np.unique(y_df)}\n')

        for domain in range(config[dataset]['num_df_domains'], config[dataset]['num_df_domains'] + config[dataset]['num_dp_domains']):
            print(f"Domain: {domain}")

            # Load synthetic data
            x_syn_dom, y_syn_dom, k_syn_dom = get_syn_data(dataset, syn_name, src_class, domain)
            print(f'x_syn_dom.shape: {x_syn_dom.shape} | np.unique(y_syn_dom): {np.unique(y_syn_dom)}')

            # Join Df and synthetic data
            x_df_syn = np.concatenate([x_df, x_syn_dom], axis=0)
            y_df_syn = np.concatenate([y_df, y_syn_dom], axis=0)
            k_df_syn = np.concatenate([k_df, k_syn_dom], axis=0)
            print(f'x_df_syn.shape: {x_df_syn.shape} | np.unique(y_df_syn): {np.unique(y_df_syn)}')

            # Load Dp data
            x_dp_dom, y_dp_dom, k_dp_dom = get_data(dataset, 'dp', src_class, domain)
            print(f'x_dp_dom.shape: {x_dp_dom.shape} | np.unique(y_dp_dom): {np.unique(y_dp_dom)}\n')

            # Train on Df plus synthetic data and test on Dp data
            print('Training on Df plus synthetic data...')
            acc, loss = train_and_test(x_df, y_df, x_dp_dom, y_dp_dom, dataset, num_epochs=num_epochs)
            save_scores(src_class, domain, acc, loss, 'FS_Df_plus_Syn', dataset)
            accs.append(acc)
            print(f'Source class: {src_class} | Domain: {domain} | Accuracy: {acc:.4f} | Loss: {loss:.4f}\n')

    print(f"Mean accuracy: {np.mean(accs):.4f}\n")




compute_TSTRFS_Df_plus_Syn('realworld_mobiact')